In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Typicals
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from time import time
import chardet

# Options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# 1. Annual data set

Open the fundamental datasets in *pandas* dataframe.

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/TFM/01. API fundamentals exports/dataset-01-05_part1.csv',sep=';')

with open('/content/drive/MyDrive/TFM/01. API fundamentals exports/dataset-01-05_part2.csv', 'rb') as f:
    result = chardet.detect(f.read())

df2 = pd.read_csv('/content/drive/MyDrive/TFM/01. API fundamentals exports/dataset-01-05_part2.csv', sep=';', encoding=result['encoding'])

df3 = pd.read_csv('/content/drive/MyDrive/TFM/01. API fundamentals exports/dataset-01-05_part3.csv',sep=';')

Now we can print the dimensions of the data set:

In [ ]:
print('Number of columns: ', df1.shape[1])
print('Number of instances in part 1: ', df1.shape[0])
print('Number of instances in part 2: ', df2.shape[0])
print('Number of instances in part 3: ', df3.shape[0])

Number of columns:  41
Number of instances in part 1:  33329
Number of instances in part 2:  29023
Number of instances in part 3:  14673


# 2. Macroeconomy data

Opening the macroeconomy dataframe.

In [ ]:
macro_annual = pd.read_csv('/content/drive/MyDrive/TFM/02. API macroeconomy data exports/macro_data.csv',sep=';')
macro_annual.rename(columns = {'Unnamed: 0':'year'}, inplace = True)

In [ ]:
print('Number of columns: ', macro_annual.shape[1])

Number of columns:  17


In [ ]:
macro_annual.iloc[-10:,:]

,year,Dif 3-Month Treasury Bill: Secondary Market Rate,Dif 10-Year Treasury Constant Maturity Rate,3-Month Treasury Bill: Secondary Market Rate,10-Year Treasury Constant Maturity Rate,dif_unemployment_rate_proc,unemployment_rate_proc,dif_employment_proc,employment_proc,dif_money_supply_proc,money_supply_proc,dif_inflation_rate_proc,inflation_rate_proc,dif_us_gdp_proc,us_gdp_proc,dif_us_population_proc,us_population_proc
80,2014,-0.025833,0.190000,0.032500,2.540833,-1.200000,6.158333,2561.166667,138919.5000,661.550000,11388.80000,-0.007452,1.007565,2268771.000,16932.05175,2268771.000,318534858.7
81,2015,0.020000,-0.405000,0.052500,2.135833,-0.883333,5.275000,2881.666667,141801.1667,656.300000,12045.10000,-0.000270,1.007295,2288042.833,17390.29525,2288042.833,320822901.5
82,2016,0.265000,-0.294167,0.317500,1.841667,-0.400000,4.875000,2531.250000,144332.4167,814.958333,12860.05833,0.013451,1.020746,2272598.833,17680.27375,2272598.833,323095500.3
83,2017,0.613333,0.488333,0.930833,2.330000,-0.516667,4.358333,2278.500000,146610.9167,730.733333,13590.79167,0.000345,1.021091,2047176.083,18076.65150,2047176.083,325142676.4
84,2018,1.008333,0.580000,1.939167,2.910000,-0.466667,3.891667,2286.083333,148897.0000,513.141667,14103.93333,-0.001989,1.019102,1739412.000,18609.07825,1739412.000,326882088.4
85,2019,0.121667,-0.765833,2.060833,2.144167,-0.208333,3.683333,1994.416667,150891.4167,714.291667,14818.22500,0.003750,1.022851,1578839.083,19036.05225,1578839.083,328460927.5
86,2020,-1.695833,-1.250000,0.365000,0.894167,4.408333,8.091667,-8738.250000,142153.1667,2832.391667,17650.61667,-0.009231,1.013620,2798197.000,18509.14275,2798197.000,331259124.5
87,2021,-0.320833,0.548333,0.044167,1.442500,-2.725000,5.366667,4127.750000,146280.9167,2875.408333,20526.02500,0.056744,1.070364,1054932.583,19609.81175,1054932.583,332314057.1
88,2022,1.977500,1.509167,2.021667,2.951667,-1.725000,3.641667,6344.333333,152625.2500,1044.958333,21570.98333,-0.005820,1.064544,1215460.083,20014.12825,1215460.083,333529517.2
89,2023,2.605000,0.695000,4.626667,3.646667,-0.141667,3.500000,2677.750000,155303.0000,-447.983333,21123.00000,-0.056549,1.007995,1684271.833,NaN,NaN,NaN


We are going to generate data for every quarter, depending on the quarter that the report has been generated.

In [ ]:
df_macro = pd.DataFrame(columns=macro_annual.columns)
df_macro['year_to_join'] = np.nan
df_macro['quarter_to_join'] = np.nan

for i in range(len(macro_annual)-1):

    for j in [0,1,2,3]:

        df_macro.loc[4*i+j, 'year_to_join'] = macro_annual.loc[i, 'year'] + 1
        df_macro.loc[4*i+j, 'quarter_to_join'] = j + 1
        df_macro.loc[4*i+j, macro_annual.columns] = macro_annual.loc[i,:]*(4-j)/4 + macro_annual.loc[i+1,:]*(j)/4

# 3. Joining frames

In [ ]:
df_list = [df1,df2,df3]
fundamentals = pd.DataFrame(columns=df1.columns)
for df in df_list:
    fundamentals = pd.concat([fundamentals, df])

In [ ]:
# Remove instances where there is no price:
fundamentals = fundamentals[fundamentals['price_adjusted'].notnull()]

# Set dates format
fundamentals['date'] = pd.to_datetime(fundamentals['date'])

<ipython-input-98-7245b38ab796>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamentals['date'] = pd.to_datetime(fundamentals['date'])


In [ ]:
tickers = pd.unique(fundamentals['Ticker']).tolist()
macro_columns = df_macro.columns

# delete the 'Unnamed' column
fundamentals = fundamentals.drop('Unnamed: 0', axis=1)

The interpolated macroeconomy dataframe in quarters, is joined with dates from the fundamental dataframe.

In [ ]:
fundamentals['year'] = fundamentals['date'].dt.year
fundamentals['quarter'] = np.nan

index_q1 = fundamentals['date'].dt.month <= 3
index_q2 = (3 < fundamentals['date'].dt.month) & ( fundamentals['date'].dt.month <= 6)
index_q3 = (6 < fundamentals['date'].dt.month) & ( fundamentals['date'].dt.month <= 9)
index_q4 = 9 < fundamentals['date'].dt.month

fundamentals.iloc[index_q1,-1] = 1
fundamentals.iloc[index_q2,-1] = 2
fundamentals.iloc[index_q3,-1] = 3
fundamentals.iloc[index_q4,-1] = 4

# Join the dataframes with the column 'year'
fundamentals = fundamentals.merge(df_macro, how='left', left_on=['year','quarter'], right_on=['year_to_join','quarter_to_join'])

In [ ]:
# delete the auxiliar columns:
fundamentals = fundamentals.drop(['year_to_join','quarter_to_join','year_x','quarter','year_y'], axis=1)
fundamentals['year'] = fundamentals['date'].dt.year

In [ ]:
fundamentals

,date,Ticker,Industry,Sector,eps,per,der,wcr,qr,fcfps,pcfr,totalAssets,intangibleAssets,otherCurrentAssets,totalLiab,otherCurrentLiab,beginPeriodCashFlow,endPeriodCashFlow,totalCashFromOperatingActivities,changeInCash,otherCashflowsFromInvestingActivities,changeInWorkingCapital,freeCashFlow,researchDevelopment,incomeBeforeTax,netIncome_x,sellingGeneralAdministrative,grossProfit,ebit,ebitda,depreciationAndAmortization,operatingIncome,totalOperatingExpenses,taxProvision,totalRevenue,costOfRevenue,dividendsPaid,commonStockSharesOutstanding,price,price_adjusted,Dif 3-Month Treasury Bill: Secondary Market Rate,Dif 10-Year Treasury Constant Maturity Rate,3-Month Treasury Bill: Secondary Market Rate,10-Year Treasury Constant Maturity Rate,dif_unemployment_rate_proc,unemployment_rate_proc,dif_employment_proc,employment_proc,dif_money_supply_proc,money_supply_proc,dif_inflation_rate_proc,inflation_rate_proc,dif_us_gdp_proc,us_gdp_proc,dif_us_population_proc,us_population_proc,year
0,2022-12-31,AA,Metals & Mining,Basic Materials,0.204437,222.857827,0.591805,1.747670,0.939747,1.839937,0.040384,1.475600e+10,29000000.0,551000000.0,8.167000e+09,1.246000e+09,1.924000e+09,1.474000e+09,822000000.0,-450000000.0,-15000000.0,-848000000.0,342000000.0,32000000.0,7.020000e+08,38000000.0,204000000.0,2.239000e+09,6.470000e+08,1.264000e+09,617000000.0,6.470000e+08,9.080000e+08,664000000.0,1.245100e+10,1.021200e+10,72000000.0,185875964.0,45.560476,45.478414,1.402917,1.268958,1.527292,2.574375,-1.975,4.072917,5790.1875,151039.166675,1502.570833,21309.743747,0.009821,1.065999,1175328.208,19913.049125,1175328.208,333225652.175,2022
1,2021-12-31,AA,Metals & Mining,Basic Materials,2.258992,23.221342,0.689854,1.559417,0.952529,2.790829,0.053202,1.341300e+10,36000000.0,372000000.0,8.741000e+09,1.473000e+09,1.610000e+09,1.924000e+09,920000000.0,314000000.0,955000000.0,-672000000.0,530000000.0,31000000.0,1.199000e+09,429000000.0,227000000.0,2.999000e+09,1.253000e+09,1.917000e+09,664000000.0,1.253000e+09,9.600000e+08,629000000.0,1.215200e+10,9.153000e+09,19000000.0,189907737.0,52.456818,51.977509,-0.664583,0.09875,0.124375,1.305417,-0.941667,6.047917,911.25,145248.9792,2864.654166,19807.172918,0.04025,1.056178,1490748.68725,19334.6445,1490748.68725,332050323.95,2021
2,2020-12-31,AA,Metals & Mining,Basic Materials,-0.075319,-298.180049,0.839976,1.637088,1.130750,0.220577,0.009821,1.486400e+10,45000000.0,290000000.0,9.873000e+09,1.356000e+09,8.830000e+08,1.610000e+09,394000000.0,727000000.0,198000000.0,-24000000.0,41000000.0,27000000.0,1.730000e+08,-14000000.0,206000000.0,1.317000e+09,4.230000e+08,1.076000e+09,653000000.0,4.230000e+08,8.860000e+08,187000000.0,9.286000e+09,7.969000e+09,20000.0,185875964.0,22.458636,22.204932,-1.241458,-1.128958,0.788958,1.206667,3.254167,6.989583,-6055.083333,144337.7292,2302.866667,16942.518753,-0.005986,1.015928,2493357.52075,18640.870125,2493357.52075,330559575.25,2020
3,2019-12-31,AA,Metals & Mining,Basic Materials,-4.598643,-4.536845,0.627878,1.377292,0.735856,1.655080,0.079330,1.464000e+10,52000000.0,288000000.0,8.784000e+09,1.078000e+09,1.116000e+09,8.830000e+08,686000000.0,-233000000.0,23000000.0,-265000000.0,307000000.0,27000000.0,-4.380000e+08,-853000000.0,280000000.0,1.896000e+09,7.140000e+08,1.427000e+09,713000000.0,7.140000e+08,1.020000e+09,415000000.0,1.043300e+10,8.537000e+09,472000000.0,185489491.0,20.863333,20.627643,0.343333,-0.429375,2.030417,2.335625,-0.272917,3.735417,2067.333334,150392.812525,664.004167,14639.652083,0.002315,1.021914,1618982.31225,18929.30875,1618982.31225,328066217.725,2019
4,2018-12-31,AA,Metals & Mining,Basic Materials,1.204026,23.383391,0.541659,1.416238,0.853032,0.259900,0.009231,1.593800e+10,57000000.0,374000000.0,8.544000e+09,8.290000e+08,1.365000e+09,1.116000e+09,448000000.0,-249000000.0,-6000000.0,-794000000.0,49000000.0,31000000.0,1.597000e+09,227000000.0,248000000.0,3.322000e+09,1.075000e+09,1.808000e+09,733000000.0,2.196000e+09,1.126000e+09,732000000.0,1.340300e+10,1.008100e+10,8270

## 3.1 Generate the differences of some key features

Generate the differences for the ratios for the companies in the S&P 500.

In [ ]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_list = np.array(sp500[0]['Symbol'])
sp500_list = sp500_list.tolist()

In [ ]:
fundamentals = fundamentals[fundamentals['Ticker'].isin(sp500_list)]

<ipython-input-63-0e35677fc1f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamentals.drop_duplicates(subset=['year', 'Ticker'], keep='first', inplace=True)


In [ ]:
fundamentals.head(50)

,date,Ticker,Industry,Sector,eps,per,der,wcr,qr,fcfps,pcfr,totalAssets,intangibleAssets,otherCurrentAssets,totalLiab,otherCurrentLiab,beginPeriodCashFlow,endPeriodCashFlow,totalCashFromOperatingActivities,changeInCash,otherCashflowsFromInvestingActivities,changeInWorkingCapital,freeCashFlow,researchDevelopment,incomeBeforeTax,netIncome_x,sellingGeneralAdministrative,grossProfit,ebit,ebitda,depreciationAndAmortization,operatingIncome,totalOperatingExpenses,taxProvision,totalRevenue,costOfRevenue,dividendsPaid,commonStockSharesOutstanding,price,price_adjusted,Dif 3-Month Treasury Bill: Secondary Market Rate,Dif 10-Year Treasury Constant Maturity Rate,3-Month Treasury Bill: Secondary Market Rate,10-Year Treasury Constant Maturity Rate,dif_unemployment_rate_proc,unemployment_rate_proc,dif_employment_proc,employment_proc,dif_money_supply_proc,money_supply_proc,dif_inflation_rate_proc,inflation_rate_proc,dif_us_gdp_proc,us_gdp_proc,dif_us_population_proc,us_population_proc,year,target,price_diff_1,price_diff_2,price_diff_3
0,2022-12-31,AAL,Airlines,Industrials,0.193857,68.140548,-3.706846,0.710318,0.604298,-1.118876,-0.084702,6.471600e+10,2.059000e+09,1.887000e+09,7.051500e+10,4.694000e+09,4.080000e+08,5.860000e+08,2.173000e+09,1.780000e+08,-1.740000e+08,-1.820000e+08,-7.330000e+08,NaN,1.860000e+08,1.270000e+08,1.815000e+09,9.037000e+09,2.304000e+09,4.446000e+09,2.142000e+09,2.304000e+09,7.274000e+09,5.900000e+07,4.897100e+10,3.993400e+10,3.549000e+09,6.551220e+08,13.209524,13.209524,1.402917,1.268958,1.527292,2.574375,-1.975,4.072917,5790.1875,151039.166675,1502.570833,21309.743747,0.009821,1.065999,1175328.208,19913.049125,1175328.208,333225652.175,2022,NaN,NaN,NaN,NaN
1,2021-12-31,AAL,Airlines,Industrials,-3.094648,-5.695169,-2.589373,0.912133,0.817689,0.453406,0.025726,6.646700e+10,1.988000e+09,1.605000e+09,7.380700e+10,4.255000e+09,3.990000e+08,4.080000e+08,7.040000e+08,9.000000e+06,-2.330000e+08,1.041000e+09,2.920000e+08,NaN,-2.548000e+09,-1.993000e+09,1.098000e+09,2.700000e+07,-5.046000e+09,1.587000e+09,6.633000e+09,-5.046000e+09,5.073000e+09,-5.550000e+08,2.988200e+10,2.985500e+10,4.047000e+09,6.440150e+08,17.624545,17.624545,-0.664583,0.09875,0.124375,1.305417,-0.941667,6.047917,911.25,145248.9792,2864.654166,19807.172918,0.04025,1.056178,1490748.68725,19334.6445,1490748.68725,332050323.95,2021,NaN,NaN,NaN,NaN
2,2020-12-31,AAL,Airlines,Industrials,-18.361687,-0.889971,-2.412844,0.669624,0.572213,-17.568115,-1.075070,6.200800e+10,2.029000e+09,1.275000e+09,6.887500e+10,4.135000e+09,2.900000e+08,3.990000e+08,-6.543000e+09,1.090000e+08,2.940000e+08,1.217000e+09,-8.501000e+09,NaN,-1.145300e+10,-8.885000e+09,5.130000e+08,-7.596000e+09,-1.042100e+10,-1.022600e+10,1.950000e+08,-1.042100e+10,3.482000e+09,-2.568000e+09,1.733700e+10,2.493300e+10,4.300000e+07,4.838880e+08,16.341364,16.341364,-1.241458,-1.128958,0.788958,1.206667,3.254167,6.989583,-6055.083333,144337.7292,2302.866667,16942.518753,-0.005986,1.015928,2493357.52075,18640.870125,2493357.52075,330559575.25,2020,NaN,NaN,NaN,NaN
3,2019-12-31,AAL,Airlines,Industrials,3.794998,7.398586,-155.177966,0.448146,0.347059,-1.019653,-0.036315,5.999500e+10,2.084000e+09,7.790000e+08,6.011300e+10,3.679000e+09,2.860000e+08,2.900000e+08,3.815000e+09,4.000000e+06,1.011000e+09,-8.670000e+08,-4.530000e+08,NaN,2.256000e+09,1.686000e+09,1.602000e+09,1.038900e+10,1.033000e+09,3.351000e+09,2.318000e+09,1.033000e+09,6.689000e+09,5.700000e+08,4.576800e+10,3.537900e+10,1.780000e+08,4.442690e+08,28.077619,27.974243,0.343333,-0.429375,2.030417,2.335625,-0.272917,3.735417,2067.333334,150392.812525,664.004167,14639.652083,0.002315,1.021914,1618982.31225,18929.30875,1618982.31225,328066217.725,2019,NaN,NaN,NaN,NaN
4,2018-12-31,AAL,Airlines,Industrials,3.032255,10.981404,-107.076923,0.477288,0.393181,-0.455268,-0.013672,6.058000e+10,2.137000e+09,6.490000e+08,6.074900e+10,3.769000e+09,3.980000e+08,2.860000e+08,3.533000e+09,-1.120000e+08,1.161000e+09,-5.660000e+08,-2.120000e+08,NaN,1.884000e+09,1.412000e+09,1

In [ ]:
# Drop the rows that are repeated:
fundamentals.drop_duplicates(subset=['year', 'Ticker'], keep='first', inplace=True)
# reset the index of the data frame
fundamentals = fundamentals.reset_index(drop=True)

In [ ]:
for row in range(len(fundamentals)):

    year = fundamentals.loc[row,'date'].year
    company = fundamentals.loc[row,'Ticker']

    # Difference in the prices:
    price_next_1 = np.nan
    price = np.nan
    price_past_1 = np.nan
    price_past_2 = np.nan
    price_past_3 = np.nan

    # Difference in eps:
    eps = np.nan
    eps_past_1 = np.nan
    eps_past_2 = np.nan
    eps_past_3 = np.nan

    # Difference in per:
    per = np.nan
    per_past_1 = np.nan
    per_past_2 = np.nan
    per_past_3 = np.nan

    # Difference in der:
    der = np.nan
    der_past_1 = np.nan
    der_past_2 = np.nan
    der_past_3 = np.nan

    # Difference in wcr:
    wcr = np.nan
    wcr_past_1 = np.nan
    wcr_past_2 = np.nan
    wcr_past_3 = np.nan

    # Difference in qr:
    qr = np.nan
    qr_past_1 = np.nan
    qr_past_2 = np.nan
    qr_past_3 = np.nan
######################################################
    try:
        price_next_1 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year+1)]['price_adjusted'])
    except:
        pass
    try:
        price = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year)]['price_adjusted'])
    except:
        pass
    try:
        price_past_1 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-1)]['price_adjusted'])
    except:
        pass
    try:
        price_past_2 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-2)]['price_adjusted'])
    except:
        pass
    try:
        price_past_3 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-3)]['price_adjusted'])
    except:
        pass
#####################################################
    try:
        eps = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year)]['eps'] )
    except:
        pass
    try:
        eps_past_1 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-1)]['eps'])
    except:
        pass
    try:
        eps_past_2 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-2)]['eps'])
    except:
        pass
    try:
        eps_past_3 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-3)]['eps'])
    except:
        pass
###################################################
    try:
        per = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year)]['per'])
    except:
        pass
    try:
        per_past_1 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-1)]['per'])
    except:
        pass
    try:
        per_past_2 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-2)]['per'])
    except:
        pass
    try:
        per_past_3 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-3)]['per'])
    except:
        pass
###################################################
    try:
        der = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year)]['der'])
    except:
        pass
    try:
        der_past_1 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-1)]['der'])
    except:
        pass
    try:
        der_past_2 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-2)]['der'])
    except:
        pass
    try:
        der_past_3 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-3)]['der'])
    except:
        pass
###################################################
    try:
        wcr = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year)]['wcr'])
    except:
        pass
    try:
        wcr_past_1 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-1)]['wcr'])
    except:
        pass
    try:
        wcr_past_2 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-2)]['wcr'])
    except:
        pass
    try:
        wcr_past_3 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-3)]['wcr'])
    except:
        pass
##################################################
    try:
        qr = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year)]['qr'])
    except:
        pass
    try:
        qr_past_1 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-1)]['qr'])
    except:
        pass
    try:
        qr_past_2 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-2)]['qr'])
    except:
        pass
    try:
        qr_past_3 = float(fundamentals[(fundamentals['Ticker']==company) & (fundamentals['year'] == year-3)]['qr'])
    except:
        pass
##################################################

    # Target variable:
    fundamentals.loc[row,'target'] = (price_next_1 - price)*100/price
    # PRICE
    fundamentals.loc[row,'price_diff_1'] = (price - price_past_1)*100/price_past_1
    fundamentals.loc[row,'price_diff_2'] = (price_past_1 - price_past_2)*100/price_past_2
    fundamentals.loc[row,'price_diff_3'] = (price_past_2 - price_past_3)*100/price_past_3
    # EPS
    fundamentals.loc[row,'eps_diff_1'] = (eps - eps_past_1)
    fundamentals.loc[row,'eps_diff_2'] = (eps_past_1 - eps_past_2)
    fundamentals.loc[row,'eps_diff_3'] = (eps_past_2 - eps_past_3)

    # PER
    fundamentals.loc[row,'per_diff_1'] = (per - per_past_1)
    fundamentals.loc[row,'per_diff_2'] = (per_past_1 - per_past_2)
    fundamentals.loc[row,'per_diff_3'] = (per_past_2 - per_past_3)

    # DER
    fundamentals.loc[row,'der_diff_1'] = (der - der_past_1)
    fundamentals.loc[row,'der_diff_2'] = (der_past_1 - der_past_2)
    fundamentals.loc[row,'der_diff_3'] = (der_past_2 - der_past_3)

    # WCR
    fundamentals.loc[row,'wcr_diff_1'] = (wcr - wcr_past_1)
    fundamentals.loc[row,'wcr_diff_2'] = (wcr_past_1 - wcr_past_2)
    fundamentals.loc[row,'wcr_diff_3'] = (wcr_past_2 - wcr_past_3)

    # QR
    fundamentals.loc[row,'qr_diff_1'] = (qr - qr_past_1)
    fundamentals.loc[row,'qr_diff_2'] = (qr_past_1 - qr_past_2)
    fundamentals.loc[row,'qr_diff_3'] = (qr_past_2 - qr_past_3)


    if row % 500 == 0:
        print('Rows calculated:', row)

Rows calculated: 0
Rows calculated: 500
Rows calculated: 1000
Rows calculated: 1500
Rows calculated: 2000
Rows calculated: 2500
Rows calculated: 3000
Rows calculated: 3500
Rows calculated: 4000
Rows calculated: 4500
Rows calculated: 5000
Rows calculated: 5500
Rows calculated: 6000
Rows calculated: 6500
Rows calculated: 7000
Rows calculated: 7500
Rows calculated: 8000
Rows calculated: 8500
Rows calculated: 9000
Rows calculated: 9500
Rows calculated: 10000
Rows calculated: 10500
Rows calculated: 11000
Rows calculated: 11500
Rows calculated: 12000
Rows calculated: 12500
Rows calculated: 13000
Rows calculated: 13500
Rows calculated: 14000
Rows calculated: 14500
Rows calculated: 15000
Rows calculated: 15500
Rows calculated: 16000
Rows calculated: 16500
Rows calculated: 17000
Rows calculated: 17500
Rows calculated: 18000
Rows calculated: 18500
Rows calculated: 19000
Rows calculated: 19500
Rows calculated: 20000
Rows calculated: 20500
Rows calculated: 21000
Rows calculated: 21500
Rows calcula

In [ ]:
fundamentals.columns

Index(['date', 'Ticker', 'Industry', 'Sector', 'eps', 'per', 'der', 'wcr',
       'qr', 'fcfps', 'pcfr', 'totalAssets', 'intangibleAssets',
       'otherCurrentAssets', 'totalLiab', 'otherCurrentLiab',
       'beginPeriodCashFlow', 'endPeriodCashFlow',
       'totalCashFromOperatingActivities', 'changeInCash',
       'otherCashflowsFromInvestingActivities', 'changeInWorkingCapital',
       'freeCashFlow', 'researchDevelopment', 'incomeBeforeTax', 'netIncome_x',
       'sellingGeneralAdministrative', 'grossProfit', 'ebit', 'ebitda',
       'depreciationAndAmortization', 'operatingIncome',
       'totalOperatingExpenses', 'taxProvision', 'totalRevenue',
       'costOfRevenue', 'dividendsPaid', 'commonStockSharesOutstanding',
       'price', 'price_adjusted',
       'Dif 3-Month Treasury Bill: Secondary Market Rate',
       'Dif 10-Year Treasury Constant Maturity Rate',
       '3-Month Treasury Bill: Secondary Market Rate',
       '10-Year Treasury Constant Maturity Rate', 'dif_unemployme

Definitive columns for the final dataset:

In [ ]:
columns_order = ['year', 'Ticker', 'Industry', 'Sector', 'price',
                 'price_adjusted', 'price_diff_1', 'price_diff_2', 'price_diff_3',
                 'eps', 'eps_diff_1', 'eps_diff_2', 'eps_diff_3',
                 'per', 'per_diff_1', 'per_diff_2', 'per_diff_3',
                 'der', 'der_diff_1', 'der_diff_2', 'der_diff_3',
                 'wcr', 'wcr_diff_1', 'wcr_diff_2', 'wcr_diff_3',
                 'qr', 'qr_diff_1', 'qr_diff_2', 'qr_diff_3',
                'fcfps', 'pcfr', 'totalAssets', 'intangibleAssets',
                'otherCurrentAssets', 'totalLiab', 'otherCurrentLiab',
                'beginPeriodCashFlow', 'endPeriodCashFlow',
                'totalCashFromOperatingActivities', 'changeInCash',
                'otherCashflowsFromInvestingActivities', 'changeInWorkingCapital',
                'freeCashFlow', 'researchDevelopment', 'incomeBeforeTax', 'netIncome_x',
                'sellingGeneralAdministrative', 'grossProfit', 'ebit', 'ebitda',
                'depreciationAndAmortization', 'operatingIncome',
                'totalOperatingExpenses', 'taxProvision', 'totalRevenue',
                'costOfRevenue', 'dividendsPaid', 'commonStockSharesOutstanding',
                'Dif 3-Month Treasury Bill: Secondary Market Rate',
                'Dif 10-Year Treasury Constant Maturity Rate',
                '3-Month Treasury Bill: Secondary Market Rate',
                '10-Year Treasury Constant Maturity Rate', 'dif_unemployment_rate_proc',
                'unemployment_rate_proc', 'dif_employment_proc', 'employment_proc',
                'dif_money_supply_proc', 'money_supply_proc', 'dif_inflation_rate_proc',
                'inflation_rate_proc', 'dif_us_gdp_proc', 'us_gdp_proc',
                'dif_us_population_proc', 'us_population_proc',
                'target']
# reorder the columns
total_dataset = fundamentals[columns_order]

In [ ]:
total_dataset

,year,Ticker,Industry,Sector,price,price_adjusted,price_diff_1,price_diff_2,price_diff_3,eps,eps_diff_1,eps_diff_2,eps_diff_3,per,per_diff_1,per_diff_2,per_diff_3,der,der_diff_1,der_diff_2,der_diff_3,wcr,wcr_diff_1,wcr_diff_2,wcr_diff_3,qr,qr_diff_1,qr_diff_2,qr_diff_3,fcfps,pcfr,totalAssets,intangibleAssets,otherCurrentAssets,totalLiab,otherCurrentLiab,beginPeriodCashFlow,endPeriodCashFlow,totalCashFromOperatingActivities,changeInCash,otherCashflowsFromInvestingActivities,changeInWorkingCapital,freeCashFlow,researchDevelopment,incomeBeforeTax,netIncome_x,sellingGeneralAdministrative,grossProfit,ebit,ebitda,depreciationAndAmortization,operatingIncome,totalOperatingExpenses,taxProvision,totalRevenue,costOfRevenue,dividendsPaid,commonStockSharesOutstanding,Dif 3-Month Treasury Bill: Secondary Market Rate,Dif 10-Year Treasury Constant Maturity Rate,3-Month Treasury Bill: Secondary Market Rate,10-Year Treasury Constant Maturity Rate,dif_unemployment_rate_proc,unemployment_rate_proc,dif_employment_proc,employment_proc,dif_money_supply_proc,money_supply_proc,dif_inflation_rate_proc,inflation_rate_proc,dif_us_gdp_proc,us_gdp_proc,dif_us_population_proc,us_population_proc,target
0,2022,AA,Metals & Mining,Basic Materials,45.560476,45.478414,-12.503667,134.080922,7.646482,0.204437,-2.054554,2.334311,4.523324,222.857827,199.636485,321.401391,-293.643204,0.591805,-0.098050,-0.150121,0.212097,1.747670,0.188253,-0.077671,0.259796,0.939747,-0.012782,-0.178221,0.394893,1.839937,0.040384,1.475600e+10,29000000.0,551000000.0,8.167000e+09,1.246000e+09,1.924000e+09,1.474000e+09,822000000.0,-450000000.0,-15000000.0,-848000000.0,342000000.0,32000000.0,7.020000e+08,38000000.0,204000000.0,2.239000e+09,6.470000e+08,1.264000e+09,617000000.0,6.470000e+08,9.080000e+08,664000000.0,1.245100e+10,1.021200e+10,72000000.0,185875964.0,1.402917,1.268958,1.527292,2.574375,-1.975,4.072917,5790.1875,151039.166675,1502.570833,21309.743747,0.009821,1.065999,1175328.208,19913.049125,1175328.208,333225652.175,NaN
1,2021,AA,Metals & Mining,Basic Materials,52.456818,51.977509,134.080922,7.646482,-25.896270,2.258992,2.334311,4.523324,-5.802669,23.221342,321.401391,-293.643204,-27.920237,0.689854,-0.150121,0.212097,0.086220,1.559417,-0.077671,0.259796,-0.038946,0.952529,-0.178221,0.394893,-0.117175,2.790829,0.053202,1.341300e+10,36000000.0,372000000.0,8.741000e+09,1.473000e+09,1.610000e+09,1.924000e+09,920000000.0,314000000.0,955000000.0,-672000000.0,530000000.0,31000000.0,1.199000e+09,429000000.0,227000000.0,2.999000e+09,1.253000e+09,1.917000e+09,664000000.0,1.253000e+09,9.600000e+08,629000000.0,1.215200e+10,9.153000e+09,19000000.0,189907737.0,-0.664583,0.09875,0.124375,1.305417,-0.941667,6.047917,911.25,145248.9792,2864.654166,19807.172918,0.04025,1.056178,1490748.68725,19334.6445,1490748.68725,332050323.95,-12.503667
2,2020,AA,Metals & Mining,Basic Materials,22.458636,22.204932,7.646482,-25.896270,-38.050433,-0.075319,4.523324,-5.802669,0.043484,-298.180049,-293.643204,-27.920237,-15.776773,0.839976,0.212097,0.086220,-0.177333,1.637088,0.259796,-0.038946,0.113040,1.130750,0.394893,-0.117175,-0.003364,0.220577,0.009821,1.486400e+10,45000000.0,290000000.0,9.873000e+09,1.356000e+09,8.830000e+08,1.610000e+09,394000000.0,727000000.0,198000000.0,-24000000.0,41000000.0,27000000.0,1.730000e+08,-14000000.0,206000000.0,1.317000e+09,4.230000e+08,1.076000e+09,653000000.0,4.230000e+08,8.860000e+08,187000000.0,9.286000e+09,7.969000e+09,20000.0,185875964.0,-1.241458,-1.128958,0.788958,1.206667,3.254167,6.989583,-6055.083333,144337.7292,2302.866667,16942.518753,-0.005986,1.015928,2493357.52075,18640.870125,2493357.52075,330559575.25,134.080922
3,2019,AA,Metals & Mining,Basic Materials,20.863333,20.627643,-25.896270,-38.050433,51.235513,-4.598643,-5.802669,0.043484,3.351864,-4.536845,-27.920237,-15.776773,52.873561,0.627878,0.086220,-0.177333,0.220053,1.377292,-0.038946,0.113040,0.175584,0.735856,-0.117175,-0.003364,0.139983,1.655080,0.079330,1.464000e+10,52000000.0,288000000.0,8.784000e

In [ ]:
total_dataset.to_excel('total_dataset_02_05.xlsx')

In [ ]:
total_dataset.to_csv('total_dataset_02_05.csv', sep=';')